In [1]:
import cymetric as cym
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.enrichment is not yet QA compliant.
  return f(*args, **kwds)


In [29]:
#load db in a evaler
with cym.dbopen('cyclus.sqlite') as db:
    evaler = cym.Evaluator(db)

In [30]:
#db.tables

In [31]:
#get transation & Agent tables
trans = evaler.eval('Transactions')
agents = evaler.eval('AgentEntry')

In [32]:
#build 2 table for SenderId and ReceiverId
agents_Sender = agents.rename(index=str, columns={'AgentId': 'SenderId'})
agents_Receiver = agents.rename(index=str, columns={'AgentId': 'ReceiverId'})

In [33]:
#Merge SenderId into Transaction
df = pd.merge(agents_Sender[['SimId', 'SenderId', 'Prototype']], trans, on=['SimId', 'SenderId'])
#rename Proto into SenderProto
df = df.rename(index=str, columns={'Prototype': 'SenderProto'})
#remove SenderId from transation table (cleaning)
df = df.drop('SenderId',1)

In [34]:
#Same for Receiver ...
df = pd.merge(agents_Receiver[['SimId', 'ReceiverId', 'Prototype']], df, on=['SimId', 'ReceiverId'])
df = df.rename(index=str, columns={'Prototype': 'ReceiverProto'})
df = df.drop('ReceiverId',1)
#df

In [35]:
#Get Ressource Table
ressource = evaler.eval('Resources')
#ressource

In [36]:
df = pd.merge(ressource[['SimId', 'ResourceId','QualId','Quantity','Units'  ]], df, on=['SimId', 'ResourceId'])


In [37]:
df = df.drop('ResourceId',1)
#df

In [38]:
trans = df[['SenderProto', 'ReceiverProto','Time', 'Quantity']].groupby(['ReceiverProto', 'SenderProto','Time'
]).sum()
trans

Quantity
ReceiverProto             SenderProto              Time                
E3_second_limited_storage E3_second_limited_source 1     1000000.000000
J1_prime_storage          PWR_separation           110      2886.203136
                                                   128      2886.203136
                                                   146      2886.203136
                                                   164      2886.203136
                                                   182      2886.203136
                                                   200      2886.203136
                                                   218      2886.203136
                                                   236      2886.203136
                                                   254      2886.203136
                                                   272      2886.203136
                                                   290      2886.203136
                                                   308      2886.203136
                                                   326      2886.203136
                                                   344      2886.203136
                                                   362      2886.203136
                                                   380      2886.203136
                                                   398      2886.203136
                                                   416      2886.203136
                                                   434      2886.203136
                                                   452      2886.203136
                                                   470      2886.203136
                                                   488      2886.203136
                                                   506      2886.203136
                                                   524      2886.203136
                                                   542      2886.203136
                                                   560      2886.203136
                                                   578      2886.203136
                                                   596      2886.203136
                                                   614      2886.203136
...                                                                 ...
waste                     PWR_separation           956      2040.477407
                                                   974      2040.477407
                                                   992      2040.477407
                                                   1010     2040.477407
                                                   1028     2040.477407
                                                   1046     2040.477407
                                                   1064     2040.477407
                                                   1082     2040.477407
                                                   1100     2040.477407
                                                   1118     2040.477407
                                                   1136     2040.477407
                                                   1154     2040.477407
                                                   1172     2040.477407
                                                   1190     2040.477407
                                                   1208     2040.477407
                                                   1226     2040.477407
                                                   1244     2040.477407
                                                   1262     2040.477407
                                                   1280     2040.477407
                                                   1298     2040.477407
                                                   1316     2040.477407
                                                   1334     2040.477407
                                                   1352     2040.477407
                               

In [39]:
toplot = trans.loc['PWR_fabrication'].loc['u_mixer']
toplot.reset_index(inplace=True)
toplot

KeyError: 'the label [u_mixer] is not in the [index]'

In [40]:
myplot = pd.DataFrame({'Time': toplot.Time/12., 'Transaction': toplot.Quantity})
myplot = myplot.groupby('Time').sum()
myplot.plot(kind='area', figsize=(10, 6))
plt.xlabel('Time [Years]')
plt.ylabel('Transaction [Kg]')
plt.savefig('test.png', dpi=326)


NameError: name 'toplot' is not defined

In [41]:
receiver = trans.index.levels[0]
receiver

Index(['E3_second_limited_storage', 'J1_prime_storage', 'J1_second_storage',
       'PWR', 'PWR_cooling', 'PWR_fabrication', 'PWR_separation',
       'PWR_storage', 'waste'],
      dtype='object', name='ReceiverProto')

In [42]:
sender = trans.index.levels[1]
sender

Index(['E3_second_limited_source', 'E3_second_limited_storage',
       'J1_second_limited_source', 'J1_second_storage', 'PWR', 'PWR_cooling',
       'PWR_fabrication', 'PWR_separation', 'PWR_storage'],
      dtype='object', name='SenderProto')

In [43]:
trans.index.levels[2]

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   23,
            ...
            1462, 1463, 1464, 1476, 1477, 1478, 1479, 1480, 1481, 1482],
           dtype='int64', name='Time', length=519)

In [44]:
df

,SimId,QualId,Quantity,Units,ReceiverProto,SenderProto,TransactionId,Commodity,Time
0,5bfd2817-8634-4e6e-a969-cc4105acc35d,5,1000000.000000,kg,E3_second_limited_storage,E3_second_limited_source,0,E3_second,1
1,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,25000.000000,kg,J1_second_storage,J1_second_limited_source,1,J1_second_initial,1
2,5bfd2817-8634-4e6e-a969-cc4105acc35d,29,25000.000000,kg,PWR_fabrication,J1_second_storage,4,J1_second_stored,2
3,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,25000.000000,kg,J1_second_storage,J1_second_limited_source,2,J1_second_initial,2
4,5bfd2817-8634-4e6e-a969-cc4105acc35d,28,5000.000000,kg,PWR_fabrication,E3_second_limited_storage,3,E3_second_stored,2
5,5bfd2817-8634-4e6e-a969-cc4105acc35d,32,29565.000000,kg,PWR,PWR_fabrication,7,PWR_MOX_new,3
6,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,25000.000000,kg,J1_second_storage,J1_second_limited_source,5,J1_second_initial,3
7,5bfd2817-8634-4e6e-a969-cc4105acc35d,30,4857.529500,kg,PWR_fabrication,E3_second_limited_storage,6,E3_second_stored,3
8,5bfd2817-8634-4e6e-a969-cc4105acc35d,29,24707.470500,kg,PWR_fabrication,J1_second_storage,8,J1_second_stored,3
9,5bfd2817-8634-4e6e-a969-cc4105acc35d,42,29565.000000,kg,PWR,PWR_fabrication,11,PWR_MOX_new,4


In [45]:
compo = evaler.eval('Compositions')
compo

,SimId,QualId,NucId,MassFrac
0,5bfd2817-8634-4e6e-a969-cc4105acc35d,1,922350000,2.500000e-03
1,5bfd2817-8634-4e6e-a969-cc4105acc35d,1,922380000,9.975000e-01
2,5bfd2817-8634-4e6e-a969-cc4105acc35d,2,922350000,7.110000e-03
3,5bfd2817-8634-4e6e-a969-cc4105acc35d,2,922380000,9.928900e-01
4,5bfd2817-8634-4e6e-a969-cc4105acc35d,3,922380000,1.000000e+00
5,5bfd2817-8634-4e6e-a969-cc4105acc35d,4,942390000,1.000000e+00
6,5bfd2817-8634-4e6e-a969-cc4105acc35d,5,922340000,2.000120e-05
7,5bfd2817-8634-4e6e-a969-cc4105acc35d,5,922350000,2.030122e-03
8,5bfd2817-8634-4e6e-a969-cc4105acc35d,5,922360000,8.300498e-04
9,5bfd2817-8634-4e6e-a969-cc4105acc35d,5,922370000,0.000000e+00


In [57]:
selected_compo = compo.loc[compo['NucId'].isin([942390000, 922380000])]
selected_compo

,SimId,QualId,NucId,MassFrac
1,5bfd2817-8634-4e6e-a969-cc4105acc35d,1,922380000,0.997500
3,5bfd2817-8634-4e6e-a969-cc4105acc35d,2,922380000,0.992890
4,5bfd2817-8634-4e6e-a969-cc4105acc35d,3,922380000,1.000000
5,5bfd2817-8634-4e6e-a969-cc4105acc35d,4,942390000,1.000000
10,5bfd2817-8634-4e6e-a969-cc4105acc35d,5,922380000,0.810869
12,5bfd2817-8634-4e6e-a969-cc4105acc35d,5,942390000,0.179775
16,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320
18,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,942390000,0.027058
23,5bfd2817-8634-4e6e-a969-cc4105acc35d,7,922380000,0.916196
25,5bfd2817-8634-4e6e-a969-cc4105acc35d,7,942390000,0.052119


In [58]:
df2 = pd.merge(selected_compo[['SimId', 'QualId','NucId','MassFrac'  ]], df, on=['SimId', 'QualId'])

In [59]:
df2

,SimId,QualId,NucId,MassFrac,Quantity,Units,ReceiverProto,SenderProto,TransactionId,Commodity,Time
0,5bfd2817-8634-4e6e-a969-cc4105acc35d,5,922380000,0.810869,1000000.000000,kg,E3_second_limited_storage,E3_second_limited_source,0,E3_second,1
1,5bfd2817-8634-4e6e-a969-cc4105acc35d,5,942390000,0.179775,1000000.000000,kg,E3_second_limited_storage,E3_second_limited_source,0,E3_second,1
2,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,25000.000000,kg,J1_second_storage,J1_second_limited_source,1,J1_second_initial,1
3,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,25000.000000,kg,J1_second_storage,J1_second_limited_source,2,J1_second_initial,2
4,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,25000.000000,kg,J1_second_storage,J1_second_limited_source,5,J1_second_initial,3
5,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,25000.000000,kg,J1_second_storage,J1_second_limited_source,9,J1_second_initial,4
6,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,24707.470500,kg,PWR_fabrication,J1_second_storage,12,J1_second_stored,4
7,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,25000.000000,kg,J1_second_storage,J1_second_limited_source,13,J1_second_initial,5
8,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,25000.000000,kg,J1_second_storage,J1_second_limited_source,17,J1_second_initial,6
9,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,25000.000000,kg,J1_second_storage,J1_second_limited_source,20,J1_second_initial,7


In [60]:
df2['Quantity'] = df2['Quantity'] * df2['MassFrac']

df2

In [61]:
df2

,SimId,QualId,NucId,MassFrac,Quantity,Units,ReceiverProto,SenderProto,TransactionId,Commodity,Time
0,5bfd2817-8634-4e6e-a969-cc4105acc35d,5,922380000,0.810869,810868.652119,kg,E3_second_limited_storage,E3_second_limited_source,0,E3_second,1
1,5bfd2817-8634-4e6e-a969-cc4105acc35d,5,942390000,0.179775,179774.786487,kg,E3_second_limited_storage,E3_second_limited_source,0,E3_second,1
2,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,23408.000000,kg,J1_second_storage,J1_second_limited_source,1,J1_second_initial,1
3,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,23408.000000,kg,J1_second_storage,J1_second_limited_source,2,J1_second_initial,2
4,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,23408.000000,kg,J1_second_storage,J1_second_limited_source,5,J1_second_initial,3
5,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,23408.000000,kg,J1_second_storage,J1_second_limited_source,9,J1_second_initial,4
6,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,23134.098779,kg,PWR_fabrication,J1_second_storage,12,J1_second_stored,4
7,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,23408.000000,kg,J1_second_storage,J1_second_limited_source,13,J1_second_initial,5
8,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,23408.000000,kg,J1_second_storage,J1_second_limited_source,17,J1_second_initial,6
9,5bfd2817-8634-4e6e-a969-cc4105acc35d,6,922380000,0.936320,23408.000000,kg,J1_second_storage,J1_second_limited_source,20,J1_second_initial,7


In [63]:
grouped_trans = df2[['ReceiverProto', 'SenderProto','Time', 'Quantity']].groupby(['ReceiverProto', 'SenderProto','Time']).sum()

In [64]:
grouped_trans

Quantity
ReceiverProto             SenderProto              Time               
E3_second_limited_storage E3_second_limited_source 1     990643.438606
J1_prime_storage          PWR_separation           110     2787.725961
                                                   128     2787.725961
                                                   146     2787.725961
                                                   164     2787.725961
                                                   182     2787.725961
                                                   200     2787.725961
                                                   218     2787.725961
                                                   236     2787.725961
                                                   254     2787.725961
                                                   272     2787.725961
                                                   290     2787.725961
                                                   308     2787.725961
                                                   326     2787.725961
                                                   344     2787.725961
                                                   362     2787.725961
                                                   380     2787.725961
                                                   398     2787.725961
                                                   416     2787.725961
                                                   434     2787.725961
                                                   452     2787.725961
                                                   470     2787.725961
                                                   488     2787.725961
                                                   506     2787.725961
                                                   524     2787.725961
                                                   542     2787.725961
                                                   560     2787.725961
                                                   578     2787.725961
                                                   596     2787.725961
                                                   614     2787.725961
...                                                                ...
waste                     PWR_separation           956      339.966582
                                                   974      339.966582
                                                   992      339.966582
                                                   1010     339.966582
                                                   1028     339.966582
                                                   1046     339.966582
                                                   1064     339.966582
                                                   1082     339.966582
                                                   1100     339.966582
                                                   1118     339.966582
                                                   1136     339.966582
                                                   1154     339.966582
                                                   1172     339.966582
                                                   1190     339.966582
                                                   1208     339.966582
                                                   1226     339.966582
                                                   1244     339.966582
                                                   1262     339.966582
                                                   1280     339.966582
                                                   1298     339.966582
                                                   1316     339.966582
                                                   1334     339.966582
                                                   1352     339.966582
                                                   1370     339.966582
               